#  **Projet Machine Learning Cyber Attack**


## Import libraries


In [64]:
import pandas as pd
import numpy as np
from uaparser import UAParser as parse_ua #import uaparser library to parse device information column
import tensorflow as tf
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder

## Import Data

Import the cybersecurity database from csv to dataframe

In [65]:
dataset_path = "cybersecurity_attacks.csv"
df = pd.read_csv(dataset_path)


Import database with the list of Os and Browser End of Care \
This database will be used to link with os and browser extracted from the user agent column (device information)\
and add the end of care date for each version of OS and Browser

In [66]:
df_browser_list = pd.read_excel('list_os_ browsers.xlsx', sheet_name='EOC BW')

In [67]:
df_OS_list = pd.read_excel('list_os_ browsers.xlsx', sheet_name='EOC OS')

## Data Engineering

### Extraction of Devices, OS and Browsers


Function used to parse the device information column and extract OS, Browser and Devices information thanks to \
    the uaparser package.

In [68]:
df['parsed_os'] = df['Device Information'].apply(lambda x: parse_ua(x).os)

Add Parsed_os column to Dataframe df this will provide all informations about the OS as \
{'name': 'Windows', 'version': '8'}


Function to concatenate the OS name with the OS version as : Windows 8

In [69]:
def os_version(row):
    '''
    Will  return a concatenation of os name and version
    '''
    ua = row['Device Information']
    parsed_ua = parse_ua(ua)
    os_name = parsed_ua.os['name']
    os_version = parsed_ua.os['version']
    version = os_version.split(".")[0] # keep just the major version ex: 8.1 -> 8
    return f"{os_name} {version}"

df['version_os'] = df.apply(os_version,axis = 1)

The following function will return same as above for the browser and the device

In [70]:
def browser_version(row):
    '''
    Will  return a concatenation of browser name and version
    '''
    ua = row['Device Information']
    parsed_ua = parse_ua(ua)
    browser_name = parsed_ua.browser['name']
    browser_version = parsed_ua.browser['version']
    version = browser_version.split(".")[0]
    return f"{browser_name} {version}"

df['browser_os'] = df.apply(browser_version,axis = 1)

In [71]:
def device_version(row):
    '''
    Will  return a concatenation of device name and version
    '''
    ua = row['Device Information']
    parsed_ua = parse_ua(ua)
    device_type = parsed_ua.device['type']
    if device_type == None:
        device_type = "computer"
    return device_type

df['device_type'] = df.apply(device_version,axis = 1)


### Extraction of the End of Care for OS and Browsers


Merge df_browser_list and df_OS_list with our df

In [72]:
df = df.merge(df_OS_list, how='left', left_on="version_os", right_on="Système d'exploitation")

In [73]:
df = df.merge(df_browser_list, how='left', left_on="browser_os", right_on= "Bibliothèque")

## Clean data creating by the above operations

Delete useless columns

In [74]:
columns_to_drop = ["parsed_os","Système d'exploitation_x","Version_x","Date de sortie_x","Système d'exploitation_y","Version_y","Date de sortie_y","Bibliothèque","Navigateur"]
df = df.drop(columns_to_drop, axis = 1)

Year extraction from the df.Timestamp column

In [75]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['year_attack'] = df['Timestamp'].dt.year

Replace NA and Inf values from the merged columns by arbitrary date as 1900\
NA values appeared by Linux OS lake of information
inf values have to be replace to avoid errors

In [76]:
df['Date de fin de support_x'] = df['Date de fin de support_x'].fillna(1900)
df['Date de fin de support_y'] = df['Date de fin de support_y'].fillna(1900)

Modify the Year values by integers or the end of care os ( support_x) and browser (support_y)
Then replace the name of columns.

In [77]:
df['Date de fin de support _OS'] = df['Date de fin de support_x'].astype(int)

In [78]:
df['Date de fin de support _Browser'] = df['Date de fin de support_y'].astype(int)

## Data Engineering II

Compute the End of Care columns for OS and Browser Columns : \
The OS and Browser version have been merge with the table df_browser_list df_OS_list to know for each when the version is not updated compared to the attack timestamp\
We create two new columns which will display if the OS and Browser are end of care : \
1 is for still update vs 0 for old version

In [79]:
df['os_EndOfCare'] = df.apply(lambda row: '1' if row['year_attack'] < row['Date de fin de support _OS'] else '0', axis=1)
df['browser_EndOfCare'] = df.apply(lambda row: '1' if row['year_attack'] < row['Date de fin de support _Browser'] else '0', axis=1)

Drop useless columns

In [80]:
columns_to_drop2 = ["Date de fin de support_x", "Date de fin de support_y", "year_attack", "Date de fin de support _OS", "Date de fin de support _Browser"]
df = df.drop(columns_to_drop2, axis = 1)

In [81]:
df.sample(20)

,Timestamp,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Payload Data,...,Geo-location Data,Proxy Information,Firewall Logs,IDS/IPS Alerts,Log Source,version_os,browser_os,device_type,os_EndOfCare,browser_EndOfCare
30554,2023-06-19 23:55:23,210.142.221.237,161.74.42.215,35166,40789,TCP,1203,Data,HTTP,Necessitatibus iusto qui dolorem officia ut. A...,...,"Ranchi, Uttarakhand",NaN,NaN,Alert Data,Server,Mac OS 10,Firefox 3,computer,0,0
14498,2021-08-12 02:34:06,56.60.1.41,190.138.226.162,57398,29594,UDP,1441,Control,FTP,Est id fuga beatae voluptatibus explicabo. Per...,...,"Chinsurah, Gujarat",NaN,Log Data,NaN,Server,Windows 95,IE 5,computer,0,0
3659,2021-07-17 06:37:11,204.208.9.108,190.162.252.153,28535,40524,TCP,1197,Data,DNS,Voluptatem numquam repudiandae alias. Reprehen...,...,"Sonipat, Tamil Nadu",122.102.35.239,NaN,NaN,Firewall,Windows NT 4,Firefox 3,computer,0,0
36572,2021-07-16 08:43:03,148.37.16.82,56.224.191.183,42114,64246,UDP,641,Data,FTP,Minus illum ut aliquam. Assumenda vitae cum ex...,...,"Dhule, Punjab",2.155.72.233,NaN,Alert Data,Server,Windows Vista,IE 5,computer,0,0
23505,2023-07-07 14:39:34,213.228.78.103,136.204.80.200,4669,19457,ICMP,76,Data,HTTP,Ipsa odio est mollitia delectus aperiam repreh...,...,"Tiruvottiyur, Gujarat",157.204.49.32,NaN,Alert Data,Server,Mac OS 10,Safari 4,computer,0,0
5216,2021-06-10 22:32:32,111.238.141.3,9.121.249.148,31393,32875,ICMP,520,Data,DNS,Fuga unde unde suscipit nesciunt. Iure repelle...,...,"Jalandhar, Tamil Nadu",NaN,NaN,NaN,Server,Linux x86_64,Firefox 3,computer,0,0
33945,2020-06-08 00:59:10,164.32.152.148,93.117.194.116,4526,7309,UDP,1013,Control,DNS,Aperiam autem repellendus nostrum ipsa. Labori...,...,"Ramgarh, Punjab",213.75.251.65,Log Data,NaN,Firewall,Windows XP,Safari 5,computer,0,0
5117,2021-11-29 13:42:11,45.55.148.14,20.21.109.166,55033,9150,ICMP,553,Control,DNS,Nesciunt quibusdam accusamus quis quo. Quaerat...,...,"Jaipur, Tripura",NaN,Log Data,NaN,Firewall,Windows 95,IE 6,computer,0,0
423,2021-12-10 09:58:17,13.223.196.90,93.57.240.43,5319,24397,TCP,915,Control,HTTP,Excepturi sint dolor ab. Ab natus excepturi at...,...,"Howrah, Nagaland",NaN,NaN,NaN,Firewall,Mac OS 10,Safari 4,computer,0,0
13130,2020-01-19 04:39:00,82.36.239.64,28.142.68.99,56942,19855,UDP,287,Data,HTTP,Temporibus placeat occaecati debitis sit deser...,...,"Ajmer, Himachal Pradesh",141.115.214.110,NaN,Alert Data,Server,Mac OS 10,Safari 5,computer,0,0
